# Clustering Validation

# External Measures

Let $\mathbf{D}$ be a dataset consiting of $n$ points $\mathbf{x}_{i}$ in  a $d$_demensional space, partitioned into $k$ clusters. Let $y_{i}\in \{1, 2, ..., k\}$ denote the ground-truth cluster membership or label information for each point. The ground-truth clustering is given as $\mathcal{T}=\{T_{1}, T_{2}, ..., T_{k}\}$, where the cluster $T_{j}$ consists of all the points with label $j$, i.e., $T_{j}=\{\mathbf{x}_{i}\in \mathbf{D}|y_{i}=j\}$. Also let $\mathcal{C}=\{C_{1}, C_{2}, ..., C{r}\}$ denote a clustering of the same dataset into $r$ clusters, obtained via some clustering algorithm, and let $\hat{y_{i}}\in \{1, 2, ..., r\}$ denote the cluster label for $\mathbf{x}_{i}$. For clarity, henceforth, we will refer to $\mathcal{T}$ as the ground-truth partitioning, and to teach $T_{i}$ as a partition. We will call $\mathcal{C}$ a clustering, with each $C_{i}$ referred to as a cluster. Because the ground truth is assumed to be known, typically clustering methods will be run with the correct number of clusters, that is, with $r=k$. However, to keep the discussion more general, we allow $r$ to be different from $k$.

External evaluation measures try capture the extent to which points from the same partition appear in the  same cluster, and the extent to which points from different partitions are grouped in different clusters. There is usually a trade-off between these two goals, which is either explicitly captured by a measure or is implicit in its computation. All of the external measures rely on the $r\times k$ contingency table $\mathbf{N}$ that is induced by a clustering $\mathcal{C}$ and the ground-truth partitioning $\mathcal{T}$, defined as follows:
$$\mathbf{N}(i, j)=n_{ij}=|C_{i}\cap T_{j}|$$
In other words, the count $n_{ij}$ denotes the number of points that are common to cluster $C_{i}$ and ground-truth partition $T_{j}$. Further, for clarity, let $n_{i}=|C_{i}|$ denote the number of points in cluster $C_{i}$, and $m_{j}=|T_{j}|$ denote the number of points in partition $T_{j}$. The contingency table can be computed from $\mathcal{T}$ and $\mathcal{C}$ in $O(n)$ time by examining the partition and cluster labels, $y_{i}$ and $\hat{y_{i}}$, for each point $\mathbf{x}_{i}\in \mathbf{D}$ and incrementing the corresponding count $n_{y_{i}\hat{y_{i}}}$ or $n_{\hat{y_{i}}y_{i}}$.

Scikit Learn Clustering Evaluation Section: https://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import adjusted_mutual_info_score
from sklearn.metrics import homogeneity_completeness_v_measure
from sklearn.metrics import fowlkes_mallows_score

In [ ]:
# Compute the purity between the clusters and given partitions
from sklearn.metrics.cluster import contingency_matrix
def purity(labels_true, labels_pred):
    '''
        input: labels_true: an array of labels of the given partitions
               labels_pred: an array of labels of the clusters
        return: the purity between the partitions and clusters
    '''
    # contingency matrix
    cmat = contingency_matrix(labels_true, labels_pred)

    return (cmat.max(axis=0) / cmat.sum()).sum()

In [ ]:
# Compute the F-measure between the clusters and given partitions
from sklearn.metrics.cluster import contingency_matrix
def fmeasure(labels_true, labels_pred):
    '''
        input: labels_true: an array of labels of the given partitions
               labels_pred: an array of labels of the clusters
        return: the F-measure between the partitions and clusters
    '''
    # contingency matrix
    cmat = contingency_matrix(labels_true, labels_pred)

    # precision for each cluster
    precisions = cmat.max(axis=0) / cmat.sum(axis=0)

    # max count for each cluster
    maxes = cmat.max(axis=0)
    # partition index for each cluster with max count
    part_idxs = cmat.argmax(axis=0)
    # partition sum for each correspoding cluster with max count
    part_sums = cmat[part_idxs].sum(axis=1)

    # recall for each cluster
    recalls = maxes / part_sums

    return (2/(1/recalls + 1/precisions)).mean()

In [ ]:
# Compute a series of clustering validation measures
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import adjusted_mutual_info_score
from sklearn.metrics import homogeneity_completeness_v_measure
from sklearn.metrics import fowlkes_mallows_score
def clustering_measures(labels_true, labels_pred):
    '''
        input: labels_true: an array of labels of the given partitions
               labels_pred: an array of labels of the clusters
        return: a dictionary contains validation measures
    '''    

    ans = {}
    ans['purity'] = purity(labels_true, labels_pred)
    ans['fMeasure'] = fmeasure(labels_true, labels_pred)

    ans['rand'] = adjusted_rand_score(labels_true, labels_pred)

    ans['mutInfo'] = adjusted_mutual_info_score(labels_true, labels_pred)

    homo, comp, vm = homogeneity_completeness_v_measure(labels_true, labels_pred)
    ans['homogeneity'] = homo
    ans['completeness'] = comp
    ans['vMeasure'] = vm

    ans['fowMal'] = fowlkes_mallows_score(labels_true, labels_pred)

    return ans